In [9]:
import os
import glob
import tiktoken
import nltk
from typing import List, Tuple

# Environment variable loader
from dotenv import load_dotenv

# Pydantic
from pydantic import BaseModel, Field

# LangChain Core
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.runnables import (
    RunnableBranch, RunnableLambda, RunnableParallel, RunnablePassthrough
)

# LangChain Document Loaders & Splitters
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, TokenTextSplitter

# LangChain Vectorstores & Embeddings
from langchain_community.vectorstores import Neo4jVector
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_openai import OpenAIEmbeddings, ChatOpenAI

# LangChain Graph
from langchain_community.graphs import Neo4jGraph
from langchain_experimental.graph_transformers import LLMGraphTransformer

# LangChain LLMs and Chains
from langchain_community.llms import Ollama
from langchain.chains.llm import LLMChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains import RetrievalQA, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

# Pinecone
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore

# Neo4j driver
from neo4j import GraphDatabase

# Sentence Transformers
from sentence_transformers import SentenceTransformer

# Hugging Face Hub
from huggingface_hub import login, hf_hub_download

# PyTorch
import torch

# Pandas
import pandas as pd

# Ragas Evaluation
from ragas import EvaluationDataset, evaluate, SingleTurnSample
from ragas.metrics import (
    LLMContextRecall,
    Faithfulness,
    FactualCorrectness,
    LLMContextPrecisionWithReference
)
from ragas.llms import LangchainLLMWrapper

# %run firstpart_baritech.ipynb  # (if needed, keep at the top of your script)


In [10]:
print(torch.__version__)

2.6.0+cu126


In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()
hf_token = os.getenv("HF_TOKEN")

from huggingface_hub import logging as hf_logging
from transformers.utils import logging as transformers_logging
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

# Enable verbose logs
hf_logging.set_verbosity_info()         # Shows download/cache actions
transformers_logging.set_verbosity_info()  # Shows model initialization logs

bge = HuggingFaceBgeEmbeddings(
    model_name="BAAI/bge-m3",
    model_kwargs={"device": "cuda", "token": hf_token},
    encode_kwargs={"normalize_embeddings": True},
    show_progress=True
)

# Trigger downloads and caching
_ = bge.embed_query("Hello world")
query_result = bge.embed_query("Hello world")
print("Length", len(query_result))


In [19]:
os.environ["PINECONE_ENVIRONMENT"] = "us-east-1"

# Fetch the API key from environment variables
pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"])
index_name = "10mnafi-rag-index"

# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=bge
)
docsearch

In [36]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":10})

llm = ChatOpenAI(
    model='gpt-4.1-mini',  # Specifying the model
    temperature=0.5,  # Controls randomness in the output
    # max_tokens=500,  # Unlimited tokens in output (or specify a max if needed)
    # timeout=None,  # Optional: Set a timeout for requests
    # max_retries=2  # Number of retries for failed requests
)

# Prompt template
system_prompt = (
    "তুমি একজন সহায়ক বাংলা সহকারী। "
    "তুমি শুধুমাত্র নিচের তথ্যসূত্র দেখে উত্তর দেবে—যদি তথ্য না থাকে, দয়া করে বলো 'তথ্য নেই'। "
    "তথ্যসূত্র:\n{context}\n"
)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}")
    ]
)

# Chains
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

# Usage
input_question = " কল্যাণীর পিতার নাম কি? "
response = rag_chain.invoke({"input": input_question})
print(f"Question : {input_question}")
print("ANSWER:", response["answer"])

# Show context chunks for debugging
retrieved_docs = retriever.invoke(input_question)

  #রবীন্দ্রনাথ ঠাকুর
#$অনুপমের ন্যায় ও হারুন মিয়ার মতো মানুষের কারণে আজও কল্যাণী ও লাবণীরা অপমানের শিকার হন– ব্যক্তিত্বের যথার্থতা নিরূপণ কর। 

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00,  3.94it/s]


Question :  কল্যাণীর পিতার নাম কি? 
ANSWER: কল্যাণীর পিতার নাম শম্ভুনাথ সেন।


Batches: 100%|██████████| 1/1 [00:00<00:00, 46.71it/s]


In [34]:
print("\n-- RETRIEVED CHUNKS --")
for idx, doc in enumerate(retrieved_docs, 1):
    print(f"Chunk {idx}:\n{doc.page_content}\nMetadata: {doc.metadata}\n")


-- RETRIEVED CHUNKS --
Chunk 1:
> **৫৩। 'একবার মামার কাছে কথাটা পাড়িয়া দেখ' - কথাটি কীসের?**  
> (ক) দানের  
> (খ) চাকরির  
> (গ) বিয়ের  
> (ঘ) ভ্রমণের  

> **৫৪। বিয়ের সময় কল্যাণীর প্রকৃত বয়স কত ছিল?**  
> (ক) ১৪ বছর  
> (খ) ১৫ বছর  
> (গ) ১৬ বছর  
> (ঘ) ১৭ বছর  
```
Metadata: {'page': 35.0, 'question': '', 'section': 'অনুপম সম্পর্কিত প্রশ্নাবলী', 'type': 'narrative'}

Chunk 2:
(গ) বিস্ময়কর  
(ঘ) সাদামাটা  

#### ২৯। কল্যাণীর পিতার নাম কী?  
(ক) হরিশচন্দ্র দত্ত  
(খ) বিনোদবিহারী সেন  
(গ) শম্ভুনাথ সেন  
(ঘ) গৌরীশংকর দত্ত  

#### ৩০। শম্ভুনাথ বাবুর বয়স কত?  
(ক) প্রায় চল্লিশ বছর  
(খ) প্রায় পঞ্চাশ বছর  
(গ) প্রায় ষাট বছর  
(ঘ) প্রায় সত্তর বছর  

#### ৩১। "তাহার বিনয়টা অজস্র নয়" - কার?  
(ক) অনুপমের  
(খ) বিনুদাদার  
(গ) শম্ভুনাথের  
(ঘ) মামার  

#### ৩২। "বাবাজি একবার এদিকে আসতে হচ্ছে" - উক্তিটি কার?  
(ক) মামার  
(খ) শম্ভুনাথের  
(গ) হরিশের  
(ঘ) মায়ের  

#### ৩৩। কষ্টিপাথর নিয়ে কে বসে ছিল?  
(ক) মামা
Metadata: {'page': 34.0, 'question': '', 'section': 'অনুপম সম্পর্কিত 